- Estimate entropy of  a single multidimensional variable in $\mathbb{R}^d$.
- On each point $\bf x_i$, center a hyperrectangle with minimal volume.
- This hyperrectangle in minimal. That is, it is constructing by including the set of 
    $\chi_i^k$ neighbors, iteratively adjusting sizes in each dimension.
- Call the sizes along each dimension $\epsilon_1, \epsilon_2, \ldots, \epsilon_d$

In [1]:
using Pkg;
Pkg.activate("../../")
using Revise, TransferEntropy

  Activating project at `~/Code/Repos/Temp/TransferEntropy.jl`


In [185]:
using StaticArrays
using DelayEmbeddings
using Entropies: Entropy, IndirectEntropy
using Neighborhood: KDTree, Chebyshev, bulkisearch, Theiler
using SpecialFunctions: digamma
import Entropies.entropy 

Base.@kwdef struct KozachenkoLeonenkoExt{B} <: IndirectEntropy
    k::Int = 1
    w::Int = 1
    base::B = MathConstants.e

    function KozachenkoLeonenkoExt(k::Int, w::Int, base::B) where B
        new{B}(k, w, base)
    end
end

"""
    volume_minirect(xᵢ, nns)

Compute the volume of the minimal enclosing rectangle with `xᵢ` at its center and 
containing all points `nᵢ ∈ nns` either within the rectangle or on one of its borders.
"""
function volume_minirect(xᵢ, nns::AbstractDataset)
    mini, maxi = minmaxima(nns)
    # dists[d] is the maximum distance from the point xᵢ to any neighbor in dimension d
    # We take twice those distances to ensure xᵢ is at the centre of the rectangle.
    dists = max.(maxi .- xᵢ, xᵢ .- mini)
    return prod(dists .* 2)
end

# Test internals here itstead of end-product, because designing an end-product test is
# a mess due to the neighbor searches. 
x = Dataset([[-1, -2], [0, -2], [3, 2]])
y = Dataset([[3, 1], [-5, 1], [3, -2]])
using Test
@test volume_minirect([0, 0], x) == 24
@test volume_minirect([0, 0], y) == 40

function mean_logvolumes(x, nn_idxs, N::Int)
    v = 0.0
    for (i, (xᵢ, nn_idxsᵢ)) in enumerate(zip(x, nn_idxs))
        nnsᵢ = @views x[nn_idxsᵢ] # the actual coordinates of the points
        v += log(MathConstants.e, volume_minirect(xᵢ, nnsᵢ))
    end
    return v / N
end
function entropy(e::KozachenkoLeonenkoExt, x::AbstractDataset{D, T}) where {D, T}
    (; k, w, base) = e
    N = length(x)
    tree = KDTree(x, Euclidean())
    nn_idxs = bulkisearch(tree, x, NeighborNumber(k), Theiler(w))
    h = digamma(N) + mean_logvolumes(x, nn_idxs, N) - digamma(k) + (D - 1)/k
    return h / log(base, MathConstants.e)
end

using Test
DN = Dataset(randn(100000, 1))
σ = 1.0
hN_base_e = 0.5 * log(MathConstants.e, 2π * σ^2) + 0.5
hN_base_2 = hN_base_e / log(2, MathConstants.e)

e_base_e = KozachenkoLeonenkoExt(k = 3, base = MathConstants.e)
e_base_2 = KozachenkoLeonenkoExt(k = 3, base = 2)

@test round(entropy(e_base_e, DN), digits = 2) == round(hN_base_e, digits = 2) 
@test round(entropy(e_base_2, DN), digits = 2) == round(hN_base_2, digits = 2) 

Test Passed

In [153]:
using Entropies
entropy(KozachenkoLeonenko(k = 5, base = MathConstants.e), pts)


2.1040992993388556

$$
\hat{H}(X) = \dfrac{1}{n}\sum_{i = 1}^n [- \log{\hat{p}(x_i)}] + \gamma
$$

$$
\begin{align}
\hat{p(x_i)} &= \log{\dfrac{1}{(n-1) \cdot r_d(x_i)^d \cdot V_d}} \\
&= \log{(1)} - \log{(n-1) \cdot r_d(x_i)^d \cdot V_d} \\
&= - [\log{(n-1) + \log(r_d(x_i)^d) + \log(V_d)}] \\
&= - \log{(n-1) - \log(r_d(x_i)^d) - \log(V_d)} \\
\end{align}
$$

So

$$
\begin{align}
\hat{H}(X) 
&= \dfrac{1}{n}\sum_{i = 1}^n [- (\log{(n-1) - \log(r_d(x_i)^d) - \log(V_d)})] + \gamma \\
&= \dfrac{1}{n}\sum_{i = 1}^n [\log{(n-1) + \log(r_d(x_i)^d) + \log(V_d)}] + \gamma \\
&= \log{(n-1) + \log(V_d) + \dfrac{1}{n}\sum_{i = 1}^n [\log(r_d(x_i)^d)}] + \gamma 
\end{align}
$$


In [182]:
XN = Dataset(randn(20000, 1));
h_XN = (0.5*log(2π) + 0.5)

h_XN_kl = entropy(Entropies.KozachenkoLeonenko(k = 1, base = MathConstants.e), XN)
h_XN_kl2 = entropy(KozachenkoLeonenko2(k = 3, base = MathConstants.e), XN)
h_XN_klext = entropy(KozachenkoLeonenkoExt(k = 3, base = MathConstants.e), XN)

@show h_XN
@show h_XN_kl
@show h_XN_kl2
@show h_XN_klext

@show h_XN * log(2.0, ℯ)
@show h_XN_kl2 * log(2.0, ℯ)
@show entropy(Entropies.KozachenkoLeonenko(k = 1, base = 2), XN)


h_XN = 1.4189385332046727
h_XN_kl = 1.415765202370185
h_XN_kl2 = 1.415765202370185
h_XN_klext = 1.4258344499628186
h_XN * log(2.0, ℯ) = 2.047095585180641
h_XN_kl2 * log(2.0, ℯ) = 2.0425174365226257
entropy(Entropies.KozachenkoLeonenko(k = 1, base = 2), XN) = 1.7869869241472909


1.7869869241472909

In [173]:
XU = Dataset(rand(20000, 1));
h_XU = 0.0
h_XU_kl = entropy(Entropies.KozachenkoLeonenko(k = 1, base = MathConstants.e), XU)
h_XU_kl2 = entropy(KozachenkoLeonenko2(k = 3, base = MathConstants.e), XU)
h_XU_klext = entropy(KozachenkoLeonenkoExt(k = 3, base = MathConstants.e), XU)

@show h_XU
@show h_XU_kl
@show h_XU_kl2
@show h_XU_klext




h_XU = 0.0
h_XU_kl = -0.010725712190660985
h_XU_kl2 = -0.010725712190660985
h_XU_klext = -0.005122087415261434


-0.005122087415261434

In [72]:
ekl = KozachenkoLeonenkoExt(k = 3, base = 2.0)
entropy(ekl, y)

InexactError: InexactError: Int64(Inf)

In [369]:
using DelayEmbeddings: minmaxima
using SpecialFunctions: digamma
using Entropies: Entropy, IndirectEntropy
using Neighborhood: KDTree, Chebyshev, bulkisearch, Theiler, NeighborNumber

export ZhuSingh

"""
    ZhuSingh <: IndirectEntropy
    ZhuSingh(k = 1, w = 0, base = MathConstants.e)

The `ZhuSingh` indirect entropy estimator (Zhu et al., 2015)[^Zhu2015] estimates the Shannon
entropy of `x` (a multi-dimensional `Dataset`) to the given `base`.

Like [`Zhu`](@ref), this estimator approximates probabilities within hyperrectangles
surrounding each point `xᵢ ∈ x` using using `k` nearest neighbor searches. However,
it also considers the number of neighbors falling on the borders of these hyperrectangles.
This estimator is an extension to the entropy estimator in Singh et al. (2003).

`w` is the Theiler window, which determines if temporal neighbors are excluded
during neighbor searches (defaults to `0`, meaning that only the point itself is excluded
when searching for neighbours).

[^Zhu2015]:
    Zhu, J., Bellanger, J. J., Shu, H., & Le Bouquin Jeannès, R. (2015). Contribution to
    transfer entropy estimation via the k-nearest-neighbors approach. Entropy, 17(6),
    4173-4201.
[^Singh2003]:
    Singh, H., Misra, N., Hnizdo, V., Fedorowicz, A., & Demchuk, E. (2003). Nearest
    neighbor estimates of entropy. American journal of mathematical and management
    sciences, 23(3-4), 301-321.
"""
Base.@kwdef struct ZhuSingh{B} <: IndirectEntropy
    k::Int = 1
    w::Int = 0
    base::B = MathConstants.e

    function ZhuSingh(k::Int, w::Int, base::B) where B
        new{B}(k, w, base)
    end
end

function entropy(e::ZhuSingh, x::AbstractDataset{D, T}) where {D, T}
    (; k, w, base) = e
    N = length(x)
    tree = KDTree(x, Euclidean())
    nn_idxs = bulkisearch(tree, x, NeighborNumber(k), Theiler(w))
    mean_logvol, mean_digammaξ = mean_logvolumes_and_digamma(x, nn_idxs, N, k)
    h = digamma(N) + mean_logvol - mean_digammaξ
    return h / log(base, MathConstants.e)
end

function mean_logvolumes_and_digamma(x, nn_idxs, N::Int, k::Int)
    T = eltype(0.0)
    logvol::T = 0.0
    digammaξ::T = 0.0
    for (i, (xᵢ, nn_idxsᵢ)) in enumerate(zip(x, nn_idxs))
        nnsᵢ = @views x[nn_idxsᵢ] # the actual coordinates of the points
        distsᵢ = maxdists(xᵢ, nnsᵢ)
        ξ = n_borderpoints(xᵢ, nnsᵢ, distsᵢ)
        digammaξ += digamma(k - ξ + 1)
        logvol += log(MathConstants.e, volume_minimal_rect(distsᵢ))
    end
    logvol /= N
    digammaξ /= N

    return logvol, digammaξ
end

"""
    volume_minimal_rect(dists) → vol

Compute the volume of a (hyper)-rectangle where the distance from its centre along the
`k`-th dimension is given by `dists[k]`, and `length(dists)` is the total dimension.
"""
volume_minimal_rect(dists) = prod(dists .* 2)

"""
    n_borderpoints(xᵢ, nns, dists) → ξ

Compute `ξ`, which is how many of `xᵢ`'s neighbor points `xⱼ ∈ nns` fall on the border of
the minimal-volume rectangle with `xᵢ` at its center.

`dists[k]` should be the maximum distance from `xᵢ[k]` to any other point along the k-th
dimension, and `length(dists)` is the total dimension.
"""
n_borderpoints(xᵢ, nns, dists) = count(any(abs.(xᵢ .- xⱼ) .== dists) for xⱼ in nns)


n_borderpoints

In [535]:
"""
    ZhuTE1 <: IndirectEntropy
    ZhuTE1(k = 1, w = 0, base = MathConstants.e)

The `ZhuTE1` transfer entropy estimator (Zhu et al., 2015)[^Zhu2015].

This estimator approximates probabilities within hyperrectangles
surrounding each point `xᵢ ∈ x` using using `k` nearest neighbor searches. However,
it also considers the number of neighbors falling on the borders of these hyperrectangles.
This estimator is an extension to the entropy estimator in Singh et al. (2003).

`w` is the Theiler window, which determines if temporal neighbors are excluded
during neighbor searches (defaults to `0`, meaning that only the point itself is excluded
when searching for neighbours).

[^Zhu2015]:
    Zhu, J., Bellanger, J. J., Shu, H., & Le Bouquin Jeannès, R. (2015). Contribution to
    transfer entropy estimation via the k-nearest-neighbors approach. Entropy, 17(6),
    4173-4201.
[^Singh2003]:
    Singh, H., Misra, N., Hnizdo, V., Fedorowicz, A., & Demchuk, E. (2003). Nearest
    neighbor estimates of entropy. American journal of mathematical and management
    sciences, 23(3-4), 301-321.
"""
Base.@kwdef struct ZhuTE1111{B} <: IndirectEntropy
    k::Int = 1
    w::Int = 0
    base::B = MathConstants.e

    function ZhuTE1111(k::Int, w::Int, base::B) where B
        k >= 2 || throw(DomainError("The number of neighbors k must be >= 2."))
        new{B}(k, w, base)
    end
end

using Neighborhood: NeighborNumber, Theiler, bulkisearch, bulksearch, WithinRange
using Distances
using DelayEmbeddings
function te(est::ZhuTE1111, 
    S::AbstractDataset{DS, Q},
    T::AbstractDataset{DT, Q}, 
    T⁺::AbstractDataset{DTT, Q}
) where {DS, DT, DTT, Q}
    println("heyo")
    (; k, w, base) = est
    joint = Dataset(S, T, T⁺)
    ST = Dataset(S, T)
    TT⁺ = Dataset(T, T⁺)
    N = length(joint)
    tree_joint = KDTree(joint, Euclidean())
    tree_ST = KDTree(ST, Euclidean())
    tree_TT⁺ = KDTree(TT⁺, Euclidean())
    tree_T = KDTree(T, Euclidean())

    nns_joint, ds_joint = bulksearch(tree_joint, joint, NeighborNumber(k), Theiler(w))
    dk = [ds_joint[i][k] for i = 1:N] # distance to k-th nearest neighbor.
    # Distances vary for each pᵢ, so can't use bulk search.
    nns_ST    = [isearch(tree_ST, pᵢ, WithinRange(dk[i])) for (i, pᵢ) in enumerate(ST)]
    nns_TT⁺   = [isearch(tree_TT⁺, pᵢ, WithinRange(dk[i])) for (i, pᵢ) in enumerate(TT⁺)]
    nns_T     = [isearch(tree_T, pᵢ, WithinRange(dk[i])) for (i, pᵢ) in enumerate(T)]

    # Find volumes
    vJ = volumes(joint, nns_joint, N)
    vST = volumes(ST, nns_joint, N)
    vTT⁺ = volumes(TT⁺, nns_joint, N)
    vT = volumes(T, nns_joint, N)
    kST = length.(nns_ST)
    kTT⁺ = length.(nns_TT⁺)
    kT = length.(nns_T)

    return mean_volumes(vJ, vST, vTT⁺, vT, N) + 
        mean_digamma(kST, kTT⁺, kT, k, N, DS, DT, DTT)
end 

function mean_volumes(vols_joint, vols_ST, vols_TT⁺, vols_T, N::Int)
    vol = 0.0
    for i = 1:N
        vol += (vols_TT⁺[i] * vols_ST[i]) / (vols_joint[i] * vols_T[i])
    end
    return vol / N
end

function mean_digamma(ks_ST, ks_TT⁺, ks_T, k::Int, N::Int, 
        dS::Int, dT::Int, dT⁺::Int)
    
    Φ = 0.0
    for i = 1:N
        Φ += digamma(k) + digamma(ks_T[i]) - digamma(ks_TT⁺[i]) - digamma(ks_ST[i]) + 
            (dT⁺ + dT - 1) / ks_TT⁺[i] + 
            (dT⁺ + dS - 1) / ks_ST[i] - 
            (dT⁺ + dT + dS - 1) / k - 
            (dT - 1) / ks_T[i]
    end
    return Φ / N
end

function volumes_and_ks(x::AbstractDataset, nn_idxs, N::Int)
    T = eltype(0.0)
    volumes = zeros(T, N)
    ks = zeros(N)
    for (i, (xᵢ, nn_idxsᵢ)) in enumerate(zip(x, nn_idxs))
        nnsᵢ = @views x[nn_idxsᵢ] # the actual coordinates of the points
        distsᵢ = maxdists(xᵢ, nnsᵢ)
        volumes[i] = volume_minimal_rect(distsᵢ)
        ks[i] = length(nn_idxsᵢ)
    end
    return volumes, ks
end

function volumes(x::AbstractDataset, nn_idxs, N::Int)
    T = eltype(0.0)
    volumes = zeros(T, N)
    for (i, (xᵢ, nn_idxsᵢ)) in enumerate(zip(x, nn_idxs))
        nnsᵢ = @views x[nn_idxsᵢ] # the actual coordinates of the points
        distsᵢ = maxdists(xᵢ, nnsᵢ)
        volumes[i] = volume_minimal_rect(distsᵢ)
    end
    return volumes
end


In [530]:
function ARM1(n)
    x = zeros(n)
    y = zeros(n)
    𝒩x = Normal(0, 0.1)
    𝒩y = Normal(0, 0.1)

    x[1:2] = rand(2)
    y[1:2] = rand(2)
    for i = 3:n
        x[i] = 0.45 * sqrt(2)*x[i-1] - 0.9*x[i-2]-0.6*y[i-2] + rand(𝒩x)
        y[i] = 0.60 * x[i-2] - 0.175*sqrt(2)*y[i-1] + 0.55*sqrt(2) * y[i-2] + rand(𝒩y)
    end

    return x, y
end


ARM1 (generic function with 1 method)

In [534]:
Ns = [round(Int, 2^i) for i = 5:0.5:11]
nreps = 10
tes = [zeros(nreps) for N in Ns]
for (k, N) in enumerate(Ns)
    for i = 1:nreps
        println("yo")
        x, y = ARM1(100)
        S = Dataset(x)[1:end-1] |> standardize
        T = Dataset(y)[1:end-1] |> standardize
        T⁺ = Dataset(y)[2:end]  |> standardize

        tes[k][i] = te(ZhuTE1111(k = 5), S, T, T⁺) / log(2, MathConstants.e)
    end
end
mean.(tes)


13-element Vector{Float64}:
 0.3120183784902036
 0.3225999093666299
 0.3275861294274093
 0.31924775981788994
 0.3198091324511628
 0.32585012656408097
 0.3341280427584144
 0.3387345875741453
 0.3266523093459226
 0.3264320742286658
 0.31961110005533755
 0.34017606758082175
 0.3266016951526681

In [491]:
# https://discourse.julialang.org/t/toeplitz-matrix/22457/3
function toeplitz(x::AbstractVector{T}) where T
    n = length(x)
    A = zeros(T, n, n)
    for i = 1:n
        for j = 1:n-i+1
            A[i,i+j-1] = x[j]
        end
        for j = n-i+2:n
            A[i, j-(n-i+1)] = x[j]
        end
    end
    return A
end

using ToeplitzMatrices
function cU(α, dU::Int)
    αs = [1; [α^k for k in 1:(dU - 1)]]
    Toeplitz(αs, αs)
end

using Distributions



3×3 Matrix{Float64}:
 1.0   0.2  0.04
 0.2   1.0  0.2
 0.04  0.2  1.0

In [496]:
Σy = cU(0.5, 3) |> Matrix
Σz = cU(0.2, 3) |> Matrix
μs = zeros(3)
Ny = MvNormal(μs, Σy)
Nz = MvNormal(μs, Σz)
Nw = Normal(0, 0.5)

Ns = [2^i for i = 8:0.5:12]

for N in Ns
    a = 1:3 |> collect

    Yₜ = rand(Ny, N)
    Zₜ = rand(Nz, N)
    Xₜ = 0.3 .* Yₜ .+ 0.1

    
end
